In [1]:
import gdown
import zipfile
import pandas as pd
import os
import re
import plotly.express as px
import statsmodels.api as sm
from pathlib import Path

In [10]:
pd.set_option('display.max_columns', None)

In [7]:
energy_data = pd.read_csv('./energy_EUI_monthly.csv')

In [8]:
energy_data.columns

Index(['simscode_clean', 'year', 'month', 'kbtu', 'buildingnumber_clean',
       'buildingname', 'grossarea', 'Monthly_EUI'],
      dtype='str')

In [9]:
master_data = pd.read_csv('./master_dataset_combined.csv')

In [11]:
master_data

,meterid,siteid,sitename,simscode,utility,readingtime,readingvalue,readingunits,readingunitsdisplay,readingwindowstart,readingwindowend,expectedwindowreadings,totalwindowreadings,missingwindowreadings,filteredwindowreadings,readingwindowsum,readingwindowmean,readingwindowstandarddeviation,readingwindowmin,readingwindowmintime,readingwindowmax,readingwindowmaxtime,year,month,day,buildingnumber,buildingname,campusname,address,city,state,postalcode,county,formalname,alsoknownas,grossarea,floorsaboveground,floorsbelowground,constructiondate,latitude,longitude
0,246162,44083,MacQuigg Laboratory,265.0,ELECTRICITY,2025-07-01T05:00:00,0.000000,kWh,Kilowatt hour,2025-07-01T05:00:00,2025-07-02T04:45:00,96,96,0,0,0.000000,0.000000,0.000000,0.000000,2025-07-01T05:00:00,0.000000,2025-07-01T05:00:00,2025,7,1,265.0,MacQuigg Laboratory (0265),Columbus,2041 College Rd,Columbus,OH,43210,Franklin,MacQuigg Laboratory,NaN,76345.0,7.0,2.0,1967-07-01,40.003534,-83.011685
1,246337,44141,Scott House,1108.0,ELECTRICITY,2025-07-01T05:00:00,18.993978,kWh,Kilowatt hour,2025-07-01T05:00:00,2025-07-02T04:45:00,96,96,0,0,1815.250848,18.908863,0.722185,17.492086,2025-07-01T18:30:00,20.569596,2025-07-02T01:30:00,2025,7,1,1108.0,Scott House (1108),Columbus,160 W Woodruff Ave,Columbus,OH,43210,Franklin,Scott House,"NRDT J, Building J - North Residential",169311.0,8.0,1.0,2015-07-01,40.004385,-83.013385
2,292885,44080,Lazenby Hall,41.0,STEAM,2025-07-01T05:00:00,0.000000,kg,Kilograms,2025-07-01T05:00:00,2025-07-02T04:45:00,96,96,0,0,5.359605,0.055829,0.063564,0.000000,2025-07-01T05:00:00,0.314916,2025-07-02T03:00:00,2025,7,1,41.0,Lazenby Hall (0041),Columbus,1827 Neil Ave Mall,Columbus,OH,43210-1261,Franklin,Lazenby Hall,NaN,69893.0,3.0,1.0,1914-07-01,39.998576,-83.015713
3,245954,44003,Baker Systems Engineering Building,280.0,ELECTRICITY,2025-07-01T05:00:00,13.799919,kWh,Kilowatt hour,2025-07-01T05:00:00,2025-07-02T04:45:00,96,96,0,0,1446.191478,15.064495,2.375681,12.502750,2025-07-02T00:15:00,20.052442,2025-07-01T18:30:00,2025,7,1,280.0,Baker Systems Engineering (0280),Columbus,1971 Neil Ave,Columbus,OH,43210-1210,Franklin,"Baker, David F, Systems Engineering Building (...",NaN,115386.0,6.0,1.0,1968-07-01,40.001596,-83.015917
4,246153,44080,Lazenby Hall,41.0,ELECTRICITY,2025-07-01T05:00:00,16.322692,kWh,Kilowatt hour,2025-07-01T05:00:00,2025-07-02T04:45:00,96,96,0,0,1936.446383,20.171316,2.280266,16.322692,2025-07-01T05:00:00,22.721327,2025-07-01T18:45:00,2025,7,1,41.0,Lazenby Hall (0041),Columbus,1827 Neil Ave Mall,Columbus,OH,43210-1261,Franklin,Lazenby Hall,NaN,69893.0,3.0,1.0,1914-07-01,39.998576,-83.015713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8964403,246049,44036,Doan Hall,89.0,ELECTRICITY,2026-01-01T04:00:00,17.875917,kWh,Kilowatt hour,2025-12-31T05:00:00,2026-01-01T04:45:00,96,96,0,0,1708.417539,17.796016,0.863724,15.965308,2025-12-31T09:00:00,18.694958,2025-12-31T10:45:00,2025,12,31,89.0,Doan Hall (0089),Medical Center,410 W 10th Ave,Columbus,OH,43210-1228,Franklin,"Doan, Charles Austin, Hall",University Hospital,684497.0,14.0,1.0,1951-06-01,39.995616,-83.017630
8964404,246351,44150,St. John Arena,76.0,ELECTRICITY,2026-01-01T04:00:00,8.304722,kWh,Kilowatt hour,2025-12-31T05:00:00,2026-01-01T04:45:00,96,96,0,0,1130.641025,11.777511,1.577607,8.303986,2026-01-01T04:15:00,15.497494,2025-12-31T18:00:00,2025,12,31,76.0,St John Arena (0076),Columbus,410 Woody Hayes Dr,Columbus,OH,43210-1166,Franklin,St John Arena,NaN,229280.0,7.0,0.0,1956-07-01,40.005332,-83.019157
8964405,246217,44104,Ohio Stadium,82.0,ELECTRICITY,2026-01-01T04:00:00,33.520429,kWh,Kilowatt hour,2025-12-31T05:00:00,2026-01-01T04:45:00,96,96,0,0,3207.761808,33.414185,0.288576,32.795463,2026-01-01T02:45:00,34.101658,2025-12-31T19:45:00,2025,12,31,82.0,Ohio Stadium (0082),Columbus,411 Woody Hayes Dr,Columbus,OH,43210,Franklin,Ohio Stadium,NaN,854223.0,10.0,0.0,1922-07-

In [12]:
master_data2 = master_data.dropna()

In [13]:
master_data2

,meterid,siteid,sitename,simscode,utility,readingtime,readingvalue,readingunits,readingunitsdisplay,readingwindowstart,readingwindowend,expectedwindowreadings,totalwindowreadings,missingwindowreadings,filteredwindowreadings,readingwindowsum,readingwindowmean,readingwindowstandarddeviation,readingwindowmin,readingwindowmintime,readingwindowmax,readingwindowmaxtime,year,month,day,buildingnumber,buildingname,campusname,address,city,state,postalcode,county,formalname,alsoknownas,grossarea,floorsaboveground,floorsbelowground,constructiondate,latitude,longitude
1,246337,44141,Scott House,1108.0,ELECTRICITY,2025-07-01T05:00:00,18.993978,kWh,Kilowatt hour,2025-07-01T05:00:00,2025-07-02T04:45:00,96,96,0,0,1815.250848,18.908863,0.722185,17.492086,2025-07-01T18:30:00,20.569596,2025-07-02T01:30:00,2025,7,1,1108.0,Scott House (1108),Columbus,160 W Woodruff Ave,Columbus,OH,43210,Franklin,Scott House,"NRDT J, Building J - North Residential",169311.0,8.0,1.0,2015-07-01,40.004385,-83.013385
11,246338,44141,Scott House,1108.0,ELECTRICITY,2025-07-01T05:00:00,24.175294,kWh,Kilowatt hour,2025-07-01T05:00:00,2025-07-02T04:45:00,96,96,0,0,2929.592731,30.516591,5.763763,23.279436,2025-07-02T04:45:00,44.616728,2025-07-01T17:00:00,2025,7,1,1108.0,Scott House (1108),Columbus,160 W Woodruff Ave,Columbus,OH,43210,Franklin,Scott House,"NRDT J, Building J - North Residential",169311.0,8.0,1.0,2015-07-01,40.004385,-83.013385
13,246579,44144,Sherman Studio Art Center,358.0,GAS,2025-07-01T05:00:00,0.000000,kWh,Kilowatt hour,2025-07-01T05:00:00,2025-07-02T04:45:00,96,96,0,0,30.402847,0.316696,2.377614,0.000000,2025-07-01T05:00:00,21.915381,2025-07-01T16:30:00,2025,7,1,358.0,Sherman Studio Art Center (0358),Columbus,1055 Carmack Rd,Columbus,OH,43210-1001,Franklin,"Sherman, Hoyt L, Studio Art Center",Foundry Metals and Glass Building,25545.0,1.0,0.0,1977-07-01,39.999738,-83.037706
18,246164,44085,Mason Hall,252.0,ELECTRICITY,2025-07-01T05:00:00,23.510315,kWh,Kilowatt hour,2025-07-01T05:00:00,2025-07-02T04:45:00,96,96,0,0,2330.598316,24.277066,1.359749,21.827992,2025-07-01T11:00:00,29.704955,2025-07-01T16:00:00,2025,7,1,252.0,Mason Hall (0252),Columbus,250 W Woodruff Ave,Columbus,OH,43210-1133,Franklin,"Mason, Raymond E, Hall",COB,68150.0,4.0,1.0,1999-07-01,40.004398,-83.015468
19,246576,44141,Scott House,1108.0,GAS,2025-07-01T05:00:00,52.336728,kWh,Kilowatt hour,2025-07-01T05:00:00,2025-07-02T04:45:00,96,96,0,0,7962.822046,82.946063,23.253160,51.996976,2025-07-01T06:00:00,163.966609,2025-07-01T10:00:00,2025,7,1,1108.0,Scott House (1108),Columbus,160 W Woodruff Ave,Columbus,OH,43210,Franklin,Scott House,"NRDT J, Building J - North Residential",169311.0,8.0,1.0,2015-07-01,40.004385,-83.013385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8964393,246424,44289,William Hall Complex - Scholars House West,847.0,ELECTRICITY,2026-01-01T04:00:00,2.929242,kWh,Kilowatt hour,2025-12-31T05:00:00,2026-01-01T04:45:00,96,96,0,0,315.965183,3.291304,0.439640,2.784131,2025-12-31T05:30:00,5.503444,2025-12-31T15:00:00,2025,12,31,847.0,William Hall Complex - Scholars House West (0847),Columbus,239 W 10th Ave,Columbus,OH,43210,Franklin,William Hall Complex - Scholars House West,William Hall Complex - Scholars House W,19381.0,3.0,1.0,2003-07-01,39.994099,-83.013515
8964397,246299,44267,Radiation Dosimetry Calibration Facility,155.0,ELECTRICITY,2026-01-01T04:00:00,0.867934,kWh,Kilowatt hour,2025-12-31T05:00:00,2026-01-01T04:45:00,96,96,0,0,87.597548,0.912474,0.145095,0.768936,2025-12-31T19:45:00,1.294948,2025-12-31T16:00:00,2025,12,31,155.0,Reactor Annex (0155),Columbus,1296 Kinnear Rd,Columbus,OH,43212-1154,Franklin,Reactor Annex,Radiation Dosimetry Calibration Facility,6237.0,1.0,0.0,1994-07-01,40.001317,-83.043761
8964398,246091,44054,Gerlach Graduate Programs Building,250.0,ELECTRICITY,2026-01-01T04:00:00,18.619672,kWh,Kilowatt hour,2025-12-31T05:00:00,2026-01-01T0

In [12]:
cols_to_use_from_df2 = [
    'simscode_clean', 'year', 'month', 
    'kbtu', 'Monthly_EUI', 'buildingnumber_clean'
]
df2_subset = energy_data[cols_to_use_from_df2]

# 2. Perform the Merge
mega_table = pd.merge(
    left=master_data,
    right=df2_subset,
    # Map the columns from Table 1 to Table 2
    left_on=['simscode', 'year', 'month'], 
    right_on=['simscode_clean', 'year', 'month'],
    how='left'
)

# 3. Cleanup (Optional)
# If you don't want the extra "clean" columns hanging around after the join:
mega_table = mega_table.drop(columns=['simscode_clean', 'buildingnumber_clean'])
mega_table = mega_table.dropna()
mega_table = mega_table.reset_index(drop=True)

In [13]:
mega_table.to_csv('./mega_table_with_EUI.csv')

In [9]:
master_data.sort_values(by=['floorsbelowground'], ascending=False)

,meterid,siteid,sitename,simscode,utility,readingtime,readingvalue,readingunits,readingunitsdisplay,readingwindowstart,readingwindowend,expectedwindowreadings,totalwindowreadings,missingwindowreadings,filteredwindowreadings,readingwindowsum,readingwindowmean,readingwindowstandarddeviation,readingwindowmin,readingwindowmintime,readingwindowmax,readingwindowmaxtime,year,month,day,buildingnumber,buildingname,campusname,address,city,state,postalcode,county,formalname,alsoknownas,grossarea,floorsaboveground,floorsbelowground,constructiondate,latitude,longitude
1673519,249679,44147,Smith Laboratory,65.0,HEAT,2025-04-08T10:00:00,20.387746,kWh,Kilowatt hour,2025-04-08T05:00:00,2025-04-09T04:45:00,96,96,0,0,1393.970425,14.520525,11.414224,5.367144,2025-04-09T03:30:00,45.470011,2025-04-08T13:15:00,2025,4,8,65.0,Smith Laboratory (0065),Columbus,174 W 18th Ave,Columbus,OH,43210-1168,Franklin,"Smith, Alpheus, Laboratory",Physics,218468.0,7.0,2.0,1950-07-01,40.002431,-83.013283
1633008,246347,44147,Smith Laboratory,65.0,ELECTRICITY,2025-04-06T18:00:00,33.529107,kWh,Kilowatt hour,2025-04-06T05:00:00,2025-04-07T04:45:00,96,96,0,0,3056.002010,31.833354,2.026464,27.387761,2025-04-06T07:00:00,35.215212,2025-04-06T11:15:00,2025,4,6,65.0,Smith Laboratory (0065),Columbus,174 W 18th Ave,Columbus,OH,43210-1168,Franklin,"Smith, Alpheus, Laboratory",Physics,218468.0,7.0,2.0,1950-07-01,40.002431,-83.013283
7843668,246347,44147,Smith Laboratory,65.0,ELECTRICITY,2025-02-14T15:00:00,33.597370,kWh,Kilowatt hour,2025-02-14T05:00:00,2025-02-15T04:45:00,96,96,0,0,3180.268648,33.127798,1.800275,29.618068,2025-02-14T06:15:00,36.683361,2025-02-14T20:00:00,2025,2,14,65.0,Smith Laboratory (0065),Columbus,174 W 18th Ave,Columbus,OH,43210-1168,Franklin,"Smith, Alpheus, Laboratory",Physics,218468.0,7.0,2.0,1950-07-01,40.002431,-83.013283
2139450,247284,44147,Smith Laboratory,65.0,COOLING,2025-04-27T10:00:00,0.000000,kWh,Kilowatt hour,2025-04-27T05:00:00,2025-04-28T04:45:00,96,96,0,0,5811.226707,60.533612,52.719723,0.000000,2025-04-27T05:00:00,146.639833,2025-04-27T21:15:00,2025,4,27,65.0,Smith Laboratory (0065),Columbus,174 W 18th Ave,Columbus,OH,43210-1168,Franklin,"Smith, Alpheus, Laboratory",Physics,218468.0,7.0,2.0,1950-07-01,40.002431,-83.013283
1408444,249679,44147,Smith Laboratory,65.0,HEAT,2025-09-27T15:00:00,4.530158,kWh,Kilowatt hour,2025-09-27T05:00:00,2025-09-28T04:45:00,96,96,0,0,194.212636,2.023048,3.551922,0.000000,2025-09-27T14:45:00,12.828396,2025-09-27T15:15:00,2025,9,27,65.0,Smith Laboratory (0065),Columbus,174 W 18th Ave,Columbus,OH,43210-1168,Franklin,"Smith, Alpheus, Laboratory",Physics,218468.0,7.0,2.0,1950-07-01,40.002431,-83.013283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,247242,44055,Goss Laboratory,180.0,COOLING,2025-08-01T05:00:00,93.916870,kWh,Kilowatt hour,2025-08-01T05:00:00,2025-08-02T04:45:00,96,96,0,0,9383.856522,97.748505,5.204832,85.081391,2025-08-01T12:00:00,106.835184,2025-08-01T15:30:00,2025,8,1,180.0,Goss Laboratory (0180),Columbus,1925 Coffey Rd,Columbus,OH,43210-1005,Franklin,"Goss, Leonard W, Laboratory",Veterinary,67943.0,4.0,0.0,1962-07-01,39.999997,-83.027551
23,247531,44117,Pfahl Executive Education Building,253.0,HEAT,2025-08-01T05:00:00,0.000000,kWh,Kilowatt hour,2025-08-01T05:00:00,2025-08-02T04:45:00,96,96,0,0,24.178358,0.251858,0.511354,0.000000,2025-08-01T05:00:00,2.063487,2025-08-01T21:45:00,2025,8,1,253.0,Pfahl Hall (0253),Columbus,280 W Woodruff Ave,Columbus,OH,43210-1133,Franklin,"Pfahl, John K, Executive Education Building",COB,58031.0,5.0,0.0,1999-07-01,40.004326,-83.016477
25,292878,44055,Goss Laboratory,180.0,STEAM,2025-08-01T05:00:00,34.096892,kg,Kilograms,2025-08-01T05:00:00,2025-08-02T04:45:00,96,96,0,0,3557.397212,37.056221,1.760940,33.649735,2025-08-01T07:30:00,41.301694,2025-08-02T02:45:00,2025,8,1,180.0,Goss Laboratory (0180),Columbus,1925 Coffey Rd,Columbus,OH,43210-1005,F

In [7]:
master_data.head()

,meterid,siteid,sitename,simscode,utility,readingtime,readingvalue,readingunits,readingunitsdisplay,readingwindowstart,readingwindowend,expectedwindowreadings,totalwindowreadings,missingwindowreadings,filteredwindowreadings,readingwindowsum,readingwindowmean,readingwindowstandarddeviation,readingwindowmin,readingwindowmintime,readingwindowmax,readingwindowmaxtime,year,month,day,buildingnumber,buildingname,campusname,address,city,state,postalcode,county,formalname,alsoknownas,grossarea,floorsaboveground,floorsbelowground,constructiondate,latitude,longitude
2,247290,44158,Torres House,1101.0,COOLING,2025-08-01T05:00:00,45.657477,kWh,Kilowatt hour,2025-08-01T05:00:00,2025-08-02T04:45:00,96,96,0,0,4958.308401,51.649046,11.584255,28.835411,2025-08-01T11:15:00,69.811928,2025-08-01T19:30:00,2025,8,1,1101.0,Torres House (1101),Columbus,187 W Lane Ave,Columbus,OH,43210,Franklin,Torres House,"NRDT A, Building A - North Residential",127073.0,9.0,0.0,2015-07-01,40.005951,-83.013655
3,246094,44055,Goss Laboratory,180.0,ELECTRICITY,2025-08-01T05:00:00,12.914948,kWh,Kilowatt hour,2025-08-01T05:00:00,2025-08-02T04:45:00,96,96,0,0,1228.310609,12.794902,0.155061,12.385272,2025-08-01T11:45:00,13.088704,2025-08-01T14:45:00,2025,8,1,180.0,Goss Laboratory (0180),Columbus,1925 Coffey Rd,Columbus,OH,43210-1005,Franklin,"Goss, Leonard W, Laboratory",Veterinary,67943.0,4.0,0.0,1962-07-01,39.999997,-83.027551
4,246515,44078,Knowlton Hall,17.0,GAS,2025-08-01T05:00:00,0.000000,kWh,Kilowatt hour,2025-08-01T05:00:00,2025-08-02T04:45:00,96,96,0,0,0.000000,0.000000,0.000000,0.000000,2025-08-01T05:00:00,0.000000,2025-08-01T05:00:00,2025,8,1,17.0,Knowlton Hall (0017),Columbus,275 W Woodruff Ave,Columbus,OH,43210-1138,Franklin,"Knowlton, Austin E, Hall","Architecture, Brown Hall, Ives",174422.0,6.0,0.0,2004-07-01,40.003599,-83.016789
5,247266,44117,Pfahl Executive Education Building,253.0,COOLING,2025-08-01T05:00:00,35.320388,kWh,Kilowatt hour,2025-08-01T05:00:00,2025-08-02T04:45:00,96,96,0,0,4513.246590,47.012985,12.880580,25.587471,2025-08-01T06:45:00,72.939052,2025-08-01T20:15:00,2025,8,1,253.0,Pfahl Hall (0253),Columbus,280 W Woodruff Ave,Columbus,OH,43210-1133,Franklin,"Pfahl, John K, Executive Education Building",COB,58031.0,5.0,0.0,1999-07-01,40.004326,-83.016477
9,292873,48593,Energy Advancement and Innovation Center,1044.0,STEAM,2025-08-01T05:00:00,1584.547901,kg,Kilograms,2025-08-01T05:00:00,2025-08-02T04:45:00,96,96,0,0,152806.072958,1591.729927,1170.181830,0.000000,2025-08-01T05:45:00,3604.544352,2025-08-01T11:45:00,2025,8,1,1044.0,Energy Advancement and Innovation Center (1044),Columbus,2281 Kenny Rd,Columbus,OH,43210,Franklin,Energy Advancement and Innovation Center,EAIC,66170.0,6.0,0.0,2023-11-03,40.005886,-83.034862


In [16]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# 1. Load the Master Dataset
df = pd.read_csv("master_dataset_combined.csv")
df['readingtime'] = pd.to_datetime(df['readingtime'])

# Detect building column
bldg_col = next((c for c in df.columns if c.lower().strip() in ['sitename', 'site name', 'building name']), None)

# ---------------------------------------------------------
# GOAL 1: Identify Inefficiency (Baseload vs Average)
# ---------------------------------------------------------
if 'readingwindowmean' in df.columns and 'readingwindowmin' in df.columns:
    # 1. Group and calculate means
    efficiency_df = df.groupby(bldg_col).agg({
        'readingwindowmean': 'mean',
        'readingwindowmin': 'mean'
    }).reset_index()

    # 2. DATA CLEANING: Remove NaNs or Infs that cause the LinAlgError
    # This specifically fixes the "SVD did not converge" error
    clean_mask = efficiency_df['readingwindowmean'].notna() & \
                 efficiency_df['readingwindowmin'].notna() & \
                 (~np.isinf(efficiency_df['readingwindowmean'])) & \
                 (~np.isinf(efficiency_df['readingwindowmin']))
    
    plot_df = efficiency_df[clean_mask].copy()

    if not plot_df.empty:
        # 3. SAFE TRENDLINE CALCULATION
        x = plot_df['readingwindowmean']
        y = plot_df['readingwindowmin']
        
        try:
            m, b = np.polyfit(x, y, 1)
            plot_df['trendline'] = m * x + b
            has_trend = True
        except np.linalg.LinAlgError:
            print("Could not converge trendline - data might be too sparse.")
            has_trend = False

        # 4. PLOTTING
        fig_efficiency = px.scatter(
            plot_df, x='readingwindowmean', y='readingwindowmin',
            hover_name=bldg_col,
            title="<b>Waste Detection: Baseload vs. Average Demand</b>",
            labels={'readingwindowmean': 'Avg Demand (kW)', 'readingwindowmin': 'Min (Baseload) (kW)'},
            template="plotly_white"
        )
        
        # Add the manual trendline if calculation succeeded
        if has_trend:
            fig_efficiency.add_trace(go.Scatter(
                x=x, y=plot_df['trendline'], 
                mode='lines', name='Trend', 
                line=dict(color='gray', dash='dot')
            ))
        
        # Add "High Waste" diagonal line (1:1 ratio)
        max_val = max(x.max(), y.max())
        fig_efficiency.add_shape(type="line", x0=0, y0=0, x1=max_val, y1=max_val, line=dict(color="Red", width=2))
        fig_efficiency.add_annotation(x=max_val*0.7, y=max_val*0.75, text="High Waste Zone", showarrow=False, font=dict(color="red"))
        
        fig_efficiency.show()
    else:
        print("No valid data points found after removing NaNs.")